In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

cust_df = pd.read_csv('./train_santander.csv', encoding='latin-1')
print('dataset shape : ', cust_df.shape)
cust_df.info()
print('\n')
# 전체 데이터에서 만족과 불만족의 비율 탐색.

print(cust_df['TARGET'].value_counts())
print('\n')

unsatisfied_cnt = cust_df[cust_df['TARGET'] == 1].TARGET.count()
print('불만족 target의 수 : ',unsatisfied_cnt)
print('\n')
total_cnt = cust_df.TARGET.count()
print('불만족의 비율 :  {:.2f}'.format(unsatisfied_cnt/total_cnt) )


# cust_df.describe()

# var3에 -999999값이 보인다. 
# 값의 개수를 조사해서 가장 높은 빈도의 수로 바꾼다.

print(cust_df[cust_df['var3'] == -999999].var3.count())

# 116개의 var3가 -999999 이다.
cust_df['var3'].replace(-999999,2,inplace=True)
cust_df.drop('ID', axis=1, inplace=True)

# 피처 세트와 레이블 세트 분리
X_features = cust_df.iloc[:,:-1]
y_labels = cust_df.iloc[:,-1]
print('피처 데이터 shape : {}'.format(X_features.shape))
print('\n')

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size = 0.2, random_state=0)

train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape : {0}, 테스트 세트 Shape : {1}'.format(X_train.shape, X_test.shape))

print('학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)
print('\n')


# 학습데이터와 테스트데이터 둘 다 4%정도가 불만족의 비율을 차지한다.

# XGBoost

# from xgboost import XGBClassifier
# from sklearn.metrics import roc_auc_score

# xgb_clf = XGBClassifier(n_estimators=500, random_state=156)

# xgb_clf.fit(X_train,y_train, early_stopping_rounds = 100, eval_metric="auc", eval_set=[(X_train,y_train), (X_test,y_test)])

# xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1], average='macro')
# print('ROC AUC : {:.4f}'.format(xgb_roc_score))



# lightGBM

from lightgbm import LGBMClassifier

# lgbm_clf = LGBMClassifier(n_estimator=500)

# evals=[(X_test, y_test)]
# lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc", eval_set = evals, verbose=True)
# lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1], average='macro')
# print('ROC AUC {:.4f}'.format(lgbm_roc_score))

# 하이퍼 파라미터 찾는 코드

from sklearn.model_selection import GridSearchCV

# lgbm_clf = LGBMClassifier(n_estimator=200)

# params= {'num_leaves' : [32,64],
#         'max_depth' : [128,160],
#         'min_child_samples' : [60,100],
#         'subsample' : [0.8,1]}

# gridcv = GridSearchCV(lgbm_clf, param_grid=params, cv=3)
# gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc", eval_set=[(X_train,y_train),(X_test,y_test)])
# print('GridSearchCV 최적 파라미터 :', gridcv.best_params_)
# lgbm_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
# print('ROC AUC {:.4f}'.format(lgbm_roc_score))

lgbm_clf = LGBMClassifier(n_estimator=1000, num_leaves=32, sumbsample=0.8, min_child_samples = 100, max_depth = 128)
evals = [(X_test,y_test)]
lgbm_clf.fit(X_train,y_train, early_stopping_rounds=100, eval_metric="auc", eval_set = evals, verbose=True)
lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC : {:.4f}'.format(lgbm_roc_score))

dataset shape :  (76020, 371)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


0    73012
1     3008
Name: TARGET, dtype: int64


불만족 target의 수 :  3008


불만족의 비율 :  0.04
116
피처 데이터 shape : (76020, 369)


학습 세트 Shape : (60816, 369), 테스트 세트 Shape : (15204, 369)
학습 세트 레이블 값 분포 비율
0    0.960964
1    0.039036
Name: TARGET, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.9583
1    0.0417
Name: TARGET, dtype: float64


[1]	valid_0's auc: 0.819488	valid_0's binary_logloss: 0.165016
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.822387	valid_0's binary_logloss: 0.159711
[3]	valid_0's auc: 0.829542	valid_0's binary_logloss: 0.156068
[4]	valid_0's auc: 0.834917	valid_0's binary_logloss: 0.153141
[5]	valid_0's auc: 0.837871	valid_0's binary_logloss: 0.150804
[6]	valid_0's auc: 0.839548	valid_0's binary_logloss: 0.148876
[7]	valid_0's auc: 0.841812	v